Research QUestions:
-------------------------

(RQ1)  Cost comparison with more extensive benchmark dataset 

(RQ2)  Which domain specific grow helps improve the overal runtime?

(RQ3)  How does extraction multiple correction subsets affect runtime

    (a) as an initial bootstrapping method;
    (b) as a replacement for grow and its complement.

(RQ4)  What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?
    
    -> What is an “interactive context”?
    -> Is the most difficult explanation faster in one over the other?
    -> Did we have to explain the previous steps as well?

In [1]:
from pathlib import Path
from pyexplain.benchmark.file_utils import *
from pyexplain.benchmark.plot import *
import IPython
import numpy as np
pd.set_option('display.max_rows', 100)

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['font.weight']= 'bold'

In [2]:
def histogram_all_explanation_times(config_expl_times: dict, order, path=None):
    #print(config_expl_times.keys())
    max_t = 0
    for label, v in config_expl_times.items():
        max_t = max([max_t, max(v)])

    bins=[0,0.1, 0.2, 0.3, 0.4, 0.5, 1,2,3,4, 5,10,20,30, 40, 50] + list(range(100, int(max_t//100 + 2)*100,100)) 
    v = []

    #fig, axs = plt.subplots(figsize=(15, 8))
    plt.figure(figsize =(15, 8))
    colors = ['r','g','b', 'c','m','y']
    
    #for o, c in zip(order, colors):
    plt.hist([config_expl_times[o] for o in order], bins=bins, label=order, stacked=True)
 
    plt.yscale('log')
    plt.xscale('log')
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.legend()
    if path:
        plt.savefig()


In [3]:
def histogram_individual_configs_explanation_times(config_expl_times: dict, order, path=None):
    #print(config_expl_times.keys())
    max_t = 0
    for label, v in config_expl_times.items():
        max_t = max([max_t, max(v)])

    bins=[0,0.1, 0.2, 0.3, 0.4, 0.5, 1,2,3,4, 5,10,20,30, 40, 50] + list(range(100, int(max_t//100 + 2)*100,100)) 
    v = []

    #fig, axs = plt.subplots(figsize=(15, 8))

    colors = ['r','g','b', 'c','m','y']
    
    fig, axs = plt.subplots(nrows=len(order)//2,ncols=2, figsize=(20, 20))
    for id, o in enumerate(order):
        row, col = id//2, id%2
        
        axs[row, col].hist(config_expl_times[o], bins=bins, label=o, stacked=True)
        axs[row, col].set_yscale('log')
        axs[row, col].set_xscale('log')
        axs[row, col].legend(fontsize=16)
        axs[row, col].tick_params(axis = 'both', labelsize = 18)
        
 
    if path:
        plt.savefig(path)


In [4]:
# MAC
environment = 'MAC'
BASE_MAC_LINUX = {
    'MAC': '/Users/emiliogamba/Documents/01_VUB/01_Research/01_Shared_Projects',
    'LINUX': '/home/emilio/research/'
}

PATH_FIGURES_POST_PAPER = Path(BASE_MAC_LINUX[environment] + "01_holygrail/latex/journal/jair21/figures/")
EXPERIMENT_RESULTS = Path(BASE_MAC_LINUX[environment] + "06_HPC_Experiments/experiments/data/output/")
BASE_OUTPUT_PATH = BASE_MAC_LINUX[environment] + "/06_HPC_Experiments/experiments/data/output/"
REMOTE_EXPERIMENT_RESULTS = "/data/brussel/101/vsc10143/hpc_experiments2/experiments/data/output/"

In [5]:
all_configs_corr_output_folders = [
    #"RQ2_LOGIC_SUDOKU_PUZZLES/2022021411/",
    #"RQ2_DEMYSTIFY_PUZZLES/2022021714/",
    "RQ3b_DEMYSTIFY_PUZZLES/2022022217/",
    "RQ3b_LOGIC_SUDOKU_PUZZLES/2022022217/",
]

path_all_corr_output_dirs = [BASE_OUTPUT_PATH + o for o in all_configs_corr_output_folders]

df_all_corr = folder_to_pandas_df_pickle(path_all_corr_output_dirs)

## These instances are ignored because their first step derives a lot 100+ literals at once, which doesn't
## represent 'good' explanations data instances
ignored_instances = [ "origin_origin.param_0_origin.param.json",
"garam_garam_9_garam-vdiff-201120.param.json",
"garam_garam_7_garam-easy-201120.param.json",
"garam_garam_5_garam-exp-201120.param.json",
"tents_tents_2_tents-1.param.json",
"tents_tents_1_tectonic-9.param.json",
"garam_garam_1_garam-adv-201120.param.json",
"garam_garam_4_garam-fiend-201120.param.json",
"tents_tents_3_tectonic-1-2-3-4-5-6-7.param.json",
"garam_garam_10_garam-diff-201120.param.json",
"pasta_pasta.param_0_pasta.param.json",
"garam_garam_8_garam-medium-201120.param.json",
"garam_garam_11_garam-tut.param.json",
"kakuro_kakuro_3_1-row-col-restrictions.param.json",
"kakuro_kakuro_0_conceptispuzzles.param.json",
"garam_garam_2_garam-master-201120.param.json",
"kakuro_kakuro_1_2-definitions-intersection.param.json",
"garam_garam_3_garam-beg-201120.param.json"]
ignored_instances += ["frietkot", "simple"]
ignored_instances += list(set(i for i in df_msg_enh["params_instance"] if 'sudoku-4x4' in i))
df_all_corr = df_all_corr[~df_all_corr["params_instance"].isin(ignored_instances)].reset_index()

df_all_corr = corr_enhance_df(df_all_corr)
df_all_corr["tot_lits_derived"] = df_all_corr.apply(lambda row: sum(row["lits_derived"]), axis=1)


## RQ4: What is the efficiency of a single step O(C)US and is single step sufficiently efficient for an interactive context?

### (a) From an incremental (random explanation?) solving point of view ?
### (b) From an instantaneous(time-to-first) explanation point of view ?


## Observation

There is only 1 best grow configuration for every explanation computer that is the best without bootstrapping , in this case:

- Greedy CorrSubsets